# Imports

We use the GatorTron model from huggingface framework

In [1]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\Users\gonca\Environments\phdenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


We use guropi's python version to solve the maximum diversity optimization problem

In [2]:
!pip install gurobipy
import torch
from torch import nn
from sklearn.decomposition import PCA
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cos
import re
import pandas as pd
from sklearn import preprocessing
import json
import pickle
import gurobipy as grb
from scipy.spatial import distance_matrix
from config import *

import random


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.3 MB ? eta -:--:--
    --------------------------------------- 0.2/10.3 MB 2.3 MB/s eta 0:00:05
   ---- ----------------------------------- 1.2/10.3 MB 11.3 MB/s eta 0:00:01
   --------------- ------------------------ 4.0/10.3 MB 23.5 MB/s eta 0:00:01
   ----------------------------- ---------- 7.5/10.3 MB 36.7 MB/s eta 0:00:01
   ---------------------------------------  10.1/10.3 MB 38.0 MB/s eta 0:00:01
   ---------------------------------------- 10.3/10.3 MB 40.9 MB/s eta 0:00:00


# Helper Functions

In [19]:
def configure_seed(seed):
  # To ensure reproducibility
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
      torch.cuda.manual_seed(seed)
      torch.backends.cudnn.deterministic = True

In [53]:
def Extract(lst,id):
    return list(list(zip(*lst))[id])

def normalizer(text_list):
  new_text_list = []
  if str(text_list) != "nan":
    for t in text_list:
      t = str(t)

      t = t.replace("[x]","[x] ")
      t = t.replace(" or ", " ").replace(" and ", " ")
      t = re.sub('[^A-Za-z0-9\(\)\[\] ]+', ' ', t)
      t = re.sub("\s\s+", " ", t)
      t = t.rstrip().lstrip()

      new_text_list.append(t)
  return new_text_list
def check_summary(text_list):
  new_text_list = []
  for t in text_list:
    if len(t) < 512:
      new_text_list.append(t)
  return new_text_list

def Qlabel_pooling(label_ids, mode):
  if mode == "max_pooling":
      label_ids = label_ids[:,1:,:]
      return nn.MaxPool1d(label_ids.shape[1],stride=label_ids.shape[1])(label_ids.permute(0,2,1)).permute(0,2,1).squeeze(1)
  elif mode == "cls_pooling":
      return label_ids[:,0,:]

def calculate_distance(element1, matrix):
    distance = 1 - torch.nn.CosineSimilarity(dim=-1)(element1.repeat(matrix.shape[0],1),matrix)
    return distance

def calculate_distance_matrix(matrix):
  new_matrix = torch.zeros((matrix.shape[0],matrix.shape[0]))
  for id,tensor_1 in enumerate(matrix):
    new_matrix[id,:] = calculate_distance(tensor_1,matrix)
  return new_matrix.cpu().numpy()


def select_max_diversity_subset(elements, m):
    n = len(elements)
    if m > n:
      return elements
    dp = [[0] * (m + 1) for _ in range(n + 1)]
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            max_without_current = dp[i - 1][j]  # Exclude the current element
            max_with_current = 0
            for k in range(i - 1, -1, -1):
                max_with_current = max(
                    max_with_current,
                    dp[k][j - 1] + calculate_distance(elements[i - 1], elements[k])
                )
            dp[i][j] = max(max_without_current, max_with_current)
    selected_subset = []
    i, j = n, m
    while i > 0 and j > 0:
        if dp[i][j] != dp[i - 1][j]:
            selected_subset.append(elements[i - 1])
            j -= 1
        i -= 1
    selected_subset.reverse()
    return selected_subset

def select_max_diversity_subset2(elements_embeddings, p):
  matrix = elements_embeddings
  d = calculate_distance_matrix(matrix)
  d = (d + d.T)/2

  N = d.shape[0]
  m = grb.Model(name="MIP Model")

  m.params.TimeLimit = 10000


  used  = [m.addVar(vtype=grb.GRB.BINARY) for i in range(N)]

  objective = grb.quicksum( d[i,j]*used[i]*used[j] for i in range(0,N) for j in range(i+1,N) )

  m.addConstr(
    lhs=grb.quicksum(used),
    sense=grb.GRB.EQUAL,
    rhs=p
  )

  # for maximization
  m.ModelSense = grb.GRB.MAXIMIZE
  m.setObjective(objective)

  # solving with Glpk
  ret = m.optimize()

  nSolutions = m.SolCount
  values = []
  for sol in range(nSolutions):
    m.setParam(grb.GRB.Param.SolutionNumber, sol)
    values.append(m.Xn)

  return np.where(np.array(values[0]) == 1)[0]

def select_max_diversity_subset3(elements_embeddings, p):
  matrix = elements_embeddings.cpu().numpy()
  d = distance_matrix(matrix, matrix, p=1)
  d = (d + d.T)/2

  N = d.shape[0]

  maxdist  = 0
  bestpair = ()
  for i in range(N):
    for j in range(i+1,N):
      if d[i,j]>maxdist:
        maxdist = d[i,j]
        bestpair = (i,j)

  P = set()
  P.add(bestpair[0])
  P.add(bestpair[1])

  while len(P)<p:
    maxdist = 0
    vbest = None
    for v in range(N):
      if v in P:
        continue
      for vprime in P:
        if d[v,vprime]>maxdist:
          maxdist = d[v,vprime]
          vbest   = v
    P.add(vbest)

  return list(P)

def free_tensor(tensor):
  tensor.detach()
  tensor.grad = None
  tensor.storage().resize_(0)
  del tensor

def merge_synonyms_from_umls_wiki(df_diag,df_proc,codes,data_umls,data_wiki):
  synonms = []
  for i in codes:
    if i in data_umls.keys():
      synonms.append((i,data_umls[i]))
    else:
      new_list = []
      aux_diag = df_diag[df_diag["icd9_code"] == i.replace(".","")]
      new_list = new_list + aux_diag["short_title"].to_list() + aux_diag["long_title"].to_list()
      aux_proc = df_proc[df_proc["icd9_code"] == i.replace(".","")]
      new_list = new_list + aux_proc["short_title"].to_list() + aux_proc["long_title"].to_list()
      synonms.append((i,[k.lower() for k in new_list]))

  code = []
  ss = []
  for s in synonms:
    code.append(s[0])
    ss.append(s[1])
  msmn_data = pd.DataFrame(data = {"code": code,"synomns":ss}).set_index("code")

  labels = codes

  if len(labels) == 50:
    richer_data = pd.DataFrame(data= {"code":labels ,"rich_data": pd.Series(data_wiki[:-1]).str.split("</s>")})
    richer_data["len"] = richer_data["rich_data"].apply(lambda x: len(x))
    rich_data_list = richer_data["rich_data"].to_list()
    new_text_list = []
    for i,text_list in enumerate(rich_data_list):
      aux = rich_data_list[:i] + rich_data_list[i+1:]
      flat_list = [item for sublist in aux for item in sublist]
      aux_list = check_summary(list(set(text_list) - set(flat_list)))
      aux_list.sort()
      new_text_list.append(aux_list)

    richer_data["rich_data"] = new_text_list
    richer_data.set_index("code",inplace = True)
    richer_data["synomns"] = msmn_data["synomns"]
    richer_data["synomns"] = richer_data["synomns"].apply(lambda x: normalizer(x))
    richer_data["len_synomns"] = richer_data["synomns"].apply(lambda x: len(x))
    richer_data["intersection_data"] = richer_data["rich_data"].apply(lambda x: set([el.lstrip().rstrip()for el in x])) - richer_data["synomns"].apply(lambda x: set(x))
    richer_data["intersection_data"] = richer_data["intersection_data"].apply(lambda x: list(x))
    richer_data["merge_data"] = richer_data["rich_data"] + richer_data["synomns"]
    richer_data["merge_data"] = richer_data["merge_data"].apply(lambda x: normalizer(x))
    richer_data["len_merge"] = richer_data["merge_data"].apply(lambda x: len(x))
    richer_data["merge_data_set"] = richer_data["merge_data"].apply(lambda x: list(set(x)))
    richer_data["merge_data_set"].apply(lambda x: x.sort())
    richer_data["len_merge_set"] = richer_data["merge_data_set"].apply(lambda x: len(x))
  else:
    richer_data = msmn_data
    richer_data.columns = ["merge_data_set"]

  synonms_merge = []
  for code,synomn in zip(richer_data.index,richer_data["merge_data_set"]):
    if len(synomn) > 200:
      synonms_merge.append(random.choices(synomn,k = 200))
    else:
      synonms_merge.append(synomn)

  return synonms_merge

# Main

In [ ]:
configure_seed(888) # Fix seed

## Gather the synonyms from each top-50/clean dataset codes

In [55]:
df_diag = pd.read_csv(MIMIC_3_DIR + "/D_ICD_DIAGNOSES.csv")
df_proc = pd.read_csv(MIMIC_3_DIR + "/D_ICD_PROCEDURES.csv")

for mode in ["50","clean"]:
  le = preprocessing.LabelEncoder()
  le = np.load(MIMIC_3_DIR + "le{}.npy".format(mode),allow_pickle=True).item()
  code_50 = list(le.classes_)

  codes = [i.replace("D","").replace("P","") for i in code_50]

  f = open(MIMIC_3_DIR + 'icd_mimic3_random_sort.json')
  data_umls = json.load(f)

  my_file = open(MIMIC_3_DIR + "labels_descriptions_wikipedia.txt", "r")
  data = my_file.read()
  data_wiki = data.split('\n')
  my_file.close()


  synonms_merge = merge_synonyms_from_umls_wiki(df_diag,df_proc,codes,data_umls,data_wiki)

  model_name = "UFNLP/gatortron-base"
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  megatron = AutoModelForSequenceClassification.from_pretrained(model_name).to(DEVICE)

  synonms_merge_matrix = []

  for s in tqdm(synonms_merge):
      s = list(s)
      if len(s) >0:
        megatron.eval()
        with torch.no_grad():
          i =  tokenizer(s ,return_tensors='pt', padding='max_length',truncation = True, max_length=512)["input_ids"].to(DEVICE)
          t =  tokenizer(s ,return_tensors='pt', padding='max_length',truncation = True, max_length=512)["token_type_ids"].to(DEVICE)
          a =  tokenizer(s ,return_tensors='pt', padding='max_length',truncation = True, max_length=512)["attention_mask"].to(DEVICE)
          out = megatron.bert(
              input_ids=i,
              token_type_ids=t,
              attention_mask=a,
              return_dict = False
          )[0]
          free_tensor(i)
          free_tensor(t)
          free_tensor(a)
          out = Qlabel_pooling(out,"cls_pooling")
          synonms_merge_matrix.append(out.clone())
          free_tensor(out)
          megatron.zero_grad()
      else:
        out = s
        synonms_merge_matrix.append(out)
      torch.cuda.empty_cache()

  with open(MIMIC_3_DIR +'/{}_synonms_matrix.pkl'.format(mode), 'wb') as f:
    pickle.dump(synonms_merge_matrix, f)

  with open(MIMIC_3_DIR +'/{}-synonms.pkl'.format(mode), 'wb') as f:
    pickle.dump(synonms_merge, f)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LabelEncoder from version 1.3.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
Some weights of MegatronBertForSequenceClassification were not initialized from the model checkpoint at /content/drive/MyDrive/Tese/MegaTron-OG_safe and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 50/50 [01:31<00:00,  1.82s/it]


## Select M synonym random

In [57]:
# ====== Random M synonyms code ========
mode = "50" # or "clean"
with open(MIMIC_3_DIR +'/{}-synonms.pkl'.format(mode), 'rb') as f:
  synonms_merge = pickle.load(f)
aux_icd_list = []
for icd_list_names in synonms_merge:
  if len(icd_list_names) == M:
    aux_icd_list.append(icd_list_names)
  elif len(icd_list_names) > M:
    aux_icd_list.append(list(np.random.choice(icd_list_names,M, replace = False)))
  elif len(icd_list_names) < M:
    aux_icd_list.append(list(np.random.choice(icd_list_names,M,replace = True)))


if mode == "50":
  with open(MIMIC_3_DIR + '/top50_MSAM{}-rand_synonyms.pkl'.format(M), 'wb') as f:
    pickle.dump(aux_icd_list, f)
else:
  with open(MIMIC_3_DIR + '/clean_MSAM{}-rand_synonyms.pkl'.format(M), 'wb') as f:
    pickle.dump(aux_icd_list, f)

## Select M synonyms by solving the MDP

In [58]:
# ======= Solve MDP ==========
mode = "50" #or "clean"

with open(MIMIC_3_DIR +'/{}_synonms_matrix.pkl'.format(mode), 'rb') as f:
  matrix = pickle.load(f)

with open(MIMIC_3_DIR +'/{}-synonms.pkl'.format(mode), 'rb') as f:
  syns = pickle.load(f)
  
M_synonyms = []
for (id,code_matrix)in enumerate(tqdm(matrix)):
  arg_list = []
  if len(code_matrix) > M:
    arg_list = select_max_diversity_subset2(code_matrix,M)

    M_synonyms.append([syns[id][k] for k in arg_list])
  else:
    if len(syns[id]) > 0:
      aux_s = syns[id].copy()
      aux_m = code_matrix.clone()
      while len(aux_s) < M:
        aux_s = aux_s + syns[id]
        aux_m = torch.cat((aux_m,code_matrix))
      arg_list = select_max_diversity_subset2(aux_m,M)
      M_synonyms.append([aux_s[k] for k in arg_list])
    else:
      M_synonyms.append(["unknown"]*M)

  if mode == "50":
    with open(MIMIC_3_DIR + '/top50_MSAM{}-MDP_synonyms.pkl'.format(M), 'wb') as f:
      pickle.dump(M_synonyms, f)
  else:
    with open(MIMIC_3_DIR + '/clean_MSAM{}-MDP_synonyms.pkl'.format(M), 'wb') as f:
      pickle.dump(M_synonyms, f)

  0%|          | 0/50 [00:00<?, ?it/s]

Restricted license - for non-production use only - expires 2025-11-24
Set parameter TimeLimit to value 10000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")



<ipython-input-53-a23928737116>:86: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  m.addConstr(


CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 36 columns and 36 nonzeros
Model fingerprint: 0x62436fe2
Model has 630 quadratic objective terms
Variable types: 0 continuous, 36 integer (36 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [1e-02, 8e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 1.1602203
Presolve time: 0.01s
Presolved: 631 rows, 666 columns, 1926 nonzeros
Variable types: 0 continuous, 666 integer (666 binary)

Root relaxation: objective 1.335114e+01, 751 iterations, 0.03 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.35114    0   36    1.16022   13.35114 

  2%|▏         | 1/50 [01:02<51:25, 62.96s/it]

...

 90%|█████████ | 45/50 [06:50<00:12,  2.56s/it]

Set parameter TimeLimit to value 10000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 4 columns and 4 nonzeros
Model fingerprint: 0x99a91c56
Model has 6 quadratic objective terms
Variable types: 0 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-07, 4e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 0.8891637
Presolve removed 1 rows and 4 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 2 available processors)

Solution count 1: 0.889164 
No other solutions better than 0.889164

Optimal solutio

 96%|█████████▌| 48/50 [06:50<00:02,  1.30s/it]

Set parameter TimeLimit to value 10000
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 1 rows, 9 columns and 9 nonzeros
Model fingerprint: 0xed8ca3b5
Model has 36 quadratic objective terms
Variable types: 0 continuous, 9 integer (9 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e-02, 5e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 4e+00]
Found heuristic solution: objective 1.0876759
Presolve time: 0.00s
Presolved: 37 rows, 45 columns, 117 nonzeros
Variable types: 0 continuous, 45 integer (45 binary)

Root relaxation: objective 2.125943e+00, 21 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  De

100%|██████████| 50/50 [06:54<00:00,  8.29s/it]
